In [1]:
import os
from pprint import pprint
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from src.grag import create_text2cypher_retriever_tool

load_dotenv(".env")

True

In [2]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

embedder_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

In [3]:
text2cypher_retriever = create_text2cypher_retriever_tool(
    neo4j_graph=neo4j_graph,
    cypher_llm=llm,
    qa_llm=llm,
    embedder_model=embedder_model,
    skip_qa_llm=False,
    verbose=True
)

In [4]:
query ="Apa isi pasal 28 UU No. 11 tahun 2008?"
# query ="Apa isi teks pasal setelah pasal 28 UU No. 11 Tahun 2008?"
# query ="Apa isi pasal 100 UU No 11 tahun 2008?"
# query = "Kenapa ikan berenang di air?"

In [5]:
result = text2cypher_retriever.invoke({"query": query})
print(result)



> Entering new GraphCypherQAChainMod chain...
Generated Cypher (1):
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'
RETURN a.text AS text

Skip QA LLM:
False

Full Context:
[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 28:\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).'}]


> Finished chain.
### **Hasil Pembuatan Kode Cypher:**
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'
RETURN a.tex

In [7]:
# Agar hasilnya dalam ToolMessage, harus invoke menggunakan schema ToolCall
# https://python.langchain.com/docs/how_to/tool_artifacts/#invoking-the-tool-with-toolcall

query ="Apa isi pasal 28 UU No. 11 tahun 2008?"
# query ="Apa isi teks pasal setelah (NEXT_ARTICLE) pasal 28 UU No. 11 Tahun 2008?"
# query ="Apa isi pasal 100 UU No 11 tahun 2008?"
# query = "Kenapa ikan berenang di air?"

text2cypher_retriever = create_text2cypher_retriever_tool(
    neo4j_graph=neo4j_graph,
    cypher_llm=llm,
    qa_llm=llm,
    embedder_model=embedder_model,
    skip_qa_llm=False,
    verbose=False
)

result = text2cypher_retriever.invoke(
    {
        "name": "text2cypher_retriever",
        "args": {"query": query},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

result.model_dump()

{'content': "### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nRETURN a.text AS text\n```\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\nPasal 28 UU No. 11 Tahun 2008 berisi tentang larangan perbuatan sebagai berikut:\n\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).",
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'tool',
 'name': 'text2cypher',
 'id': None,
 'tool_call_id': '123',
 'artifact': {'is_context_fetched': True,
  'cypher_gen_usage_metadata': {'input_tokens': 2805,
   'output_tokens': 65,
  

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent


text2cypher_retriever = create_text2cypher_retriever_tool(
    neo4j_graph=neo4j_graph,
    cypher_llm=llm,
    qa_llm=llm,
    embedder_model=embedder_model,
    skip_qa_llm=False,
    verbose=False
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

checkpointer = MemorySaver()
config = {"configurable": {"thread_id": "1"}}

react_agent = create_react_agent(model=llm, tools=[text2cypher_retriever], checkpointer=checkpointer, version="v2")
print(react_agent.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
          +-------+           
          | agent |           
          +-------+           
         *         .          
       **           ..        
      *               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


In [9]:
query ="Apa isi pasal 28 UU No. 11 tahun 2008?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Wajib gunakan tool text2cypher")},
    config=config
)

result["messages"]

[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [10]:
query ="Bagaimana dengan isi teks pasal setelah pasal 28 UU No. 11 Tahun 2008?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]

[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [11]:
query = "Tadi isi pasal 28 apa? Aku lupa"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Ngga perlu pakai tool call lagi ya")},
    config=config
)

result["messages"]

[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [12]:
query ="Kalau isi pasal 100 apa ya?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]

[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [13]:
query ="Memangnya berapa total jumlah pasal di dalam UU no 11 tahun 2008?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]


[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [14]:
query = "Ohh begitu, memangnya kenapa ikan berenang di air?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]

[HumanMessage(content='Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool text2cypher', additional_kwargs={}, response_metadata={}, id='e2f6d50f-0d2a-4399-8edc-96ee31bcf3e0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "Apa isi pasal 28 UU No. 11 tahun 2008?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-87ce3640-33c7-4402-8f86-f92fbf086ced-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'Apa isi pasal 28 UU No. 11 tahun 2008?'}, 'id': '10d811da-e550-46ae-ac0c-2f49304cca9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 24, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'\nR

In [15]:
pprint(result["messages"][3].model_dump())

{'additional_kwargs': {},
 'content': 'Pasal 28 UU No. 11 Tahun 2008 berisi tentang:\n'
            '\n'
            '(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita '
            'bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam '
            'Transaksi Elektronik.\n'
            '\n'
            '(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan '
            'informasi yang ditujukan untuk menimbulkan rasa kebencian atau '
            'permusuhan individu dan/atau kelompok masyarakat tertentu '
            'berdasarkan atas suku, agama, ras, dan antargolongan (SARA).',
 'example': False,
 'id': 'run-9ae9dba8-d028-473a-a5bb-500faa495b5b-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'STOP',
                       'prompt_feedback': {'block_reason': 0,
                                           'safety_ratings': []},
                       'safety_ratings': []},
 'tool_calls': [],
 'type': 'ai',
 'usage_m

In [16]:
from langchain_core.messages import AIMessage

ai_results = []

for message in result["messages"]:
    if isinstance(message, HumanMessage):
        query = message.content
    if isinstance(message, AIMessage):
        # ai_results.append(message)
        if message.tool_calls:
            ai_results.append({
                "type": "tool_call",
                "query": query,
                "usage": message.usage_metadata
            })
        else:
            ai_results.append({
                "type": "model_response",
                "query": query,
                "response": message.content,
                "usage": message.usage_metadata
            })
                

for ai_result in ai_results:
    pprint(ai_result)
    print()

{'query': 'Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool '
          'text2cypher',
 'type': 'tool_call',
 'usage': {'input_token_details': {'cache_read': 0},
           'input_tokens': 55,
           'output_tokens': 24,
           'total_tokens': 79}}

{'query': 'Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool '
          'text2cypher',
 'response': 'Pasal 28 UU No. 11 Tahun 2008 berisi tentang:\n'
             '\n'
             '(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita '
             'bohong dan menyesatkan yang mengakibatkan kerugian konsumen '
             'dalam Transaksi Elektronik.\n'
             '\n'
             '(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan '
             'informasi yang ditujukan untuk menimbulkan rasa kebencian atau '
             'permusuhan individu dan/atau kelompok masyarakat tertentu '
             'berdasarkan atas suku, agama, ras, dan antargolongan (SARA).',
 'type': 'model_response',
 'usa

In [17]:
result['messages'][2].model_dump()['artifact']

{'is_context_fetched': True,
 'cypher_gen_usage_metadata': {'input_tokens': 2805,
  'output_tokens': 65,
  'total_tokens': 2870},
 'qa_usage_metadata': {'input_tokens': 290,
  'output_tokens': 98,
  'total_tokens': 388},
 'run_time': 2.4130427837371826}

In [18]:
from langchain_core.messages import AIMessage
from langchain_core.messages import ToolMessage

tool_results = []

for message in result["messages"]:
    if isinstance(message, HumanMessage):
        query = message.content
    if isinstance(message, ToolMessage):
        tool_results.append({
            "type": "text2cypher",
            "query": query,
            "usage": message.artifact
        })
                

for tool_result in tool_results:
    pprint(tool_result)
    print()

{'query': 'Apa isi pasal 28 UU No. 11 tahun 2008? Wajib gunakan tool '
          'text2cypher',
 'type': 'text2cypher',
 'usage': {'cypher_gen_usage_metadata': {'input_tokens': 2805,
                                         'output_tokens': 65,
                                         'total_tokens': 2870},
           'is_context_fetched': True,
           'qa_usage_metadata': {'input_tokens': 290,
                                 'output_tokens': 98,
                                 'total_tokens': 388},
           'run_time': 2.4130427837371826}}

{'query': 'Bagaimana dengan isi teks pasal setelah pasal 28 UU No. 11 Tahun '
          '2008?',
 'type': 'text2cypher',
 'usage': {'cypher_gen_usage_metadata': {'input_tokens': 2807,
                                         'output_tokens': 77,
                                         'total_tokens': 2884},
           'is_context_fetched': True,
           'qa_usage_metadata': {'input_tokens': 248,
                                 'output_